In [ ]:
import pandas as pd

In [ ]:
#Datenimport
oxygenrawDF = pd.read_csv('../data/Katalysator.txt',sep='\t')
oxygenrawDF

In [ ]:
argonrawDF = pd.read_csv('../data/KatalysatorArgon.txt',sep='\t')
argonrawDF

In [ ]:
#Entfernung nicht relevanter Daten
argonDF=argonrawDF.loc[argonrawDF['cycle number']==3]
argonDF

In [ ]:
#Index bei 0 starten lassen
argonDF.reset_index(level=None, drop=True, inplace=True)
argonDF

In [ ]:
oxygenDF=oxygenrawDF.loc[oxygenrawDF['cycle number']==3]
oxygenDF

In [ ]:
oxygenDF.reset_index(level=None, drop=True, inplace=True)
oxygenDF

In [ ]:
#Wert Referenzelektrode [V]
E_ref=0.920
#Wert ohmscher Widerstand GC [Ohm]
E_ohm=48
#Masse Katalysator [mg]
m=0.05975

In [ ]:
#Zeit des ersten Datenpunktes null setzen
oxygenminTime=oxygenDF.loc[0, 'time/s']
oxygenDF['time/s']=oxygenDF['time/s']-oxygenminTime
oxygenDF

In [ ]:
argonminTime=argonDF.loc[0, 'time/s']
argonDF['time/s']=argonDF['time/s']-argonminTime
argonDF

In [ ]:
import numpy as np
from scipy import interpolate

In [ ]:
#argontck = interpolate.splrep(argonDF['time/s'], argonDF['<I>/mA'], s=0)
#argontnew = np.arange(0, argonDF.loc[601,'time/s'], 0.666)
#argonInew = interpolate.splev(argontnew, argontck, der=0)

In [ ]:
#Interpolation der Sauerstoffdaten, Werte für Zeitpunkte der Argonreihe ausgeben
oxygentck = interpolate.splrep(oxygenDF['time/s'], oxygenDF['<I>/mA'], s=0)
oxygenInterI=interpolate.splev(argonDF['time/s'],oxygentck, der=0)

In [ ]:
#interpolierte Werte der Tabelle hinzufügen
argonDF['<I>Oxygen/mA']=oxygenInterI.tolist()
argonDF
argonDF=argonDF.rename(columns={'<I>/mA':'<I>Argon/mA'})
argonDF

In [ ]:
#Vertauschen von Spalten und Berechnung der tatsächlichen Aktivität
argonDF=argonDF.reindex(columns=['time/s', 'Ewe/V', 'cycle number', '<I>Oxygen/mA', '<I>Argon/mA'])
argonDF
argonDF['<I>Oxygenreal/mA']=argonDF['<I>Oxygen/mA']-argonDF['<I>Argon/mA']
argonDF

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axis =plt.subplots(1,1)
axis.plot(argonDF['Ewe/V'],argonDF['<I>Oxygenreal/mA'], linewidth=0.5, label='Oxygen, real')
axis.plot(argonDF['Ewe/V'],argonDF['<I>Oxygen/mA'], linewidth=0.5, label='Oxygen')
axis.plot(argonDF['Ewe/V'],argonDF['<I>Argon/mA'], linewidth=0.5, label='Argon')
axis.set_xlabel('E_we [V]')
axis.set_ylabel('I [mA]')
axis.legend()
#axis.set_facecolor('white')
#Graphexport
#fig.savefig('Data/comparison.pdf', facecolor=axis.get_facecolor(), edgecolor='none')
fig.savefig('../data/Katalysator-Comparison.pdf')

In [ ]:
#Massennormierung des Stroms
argonDF['<I>Oxygenreal_norm/mA/mg']=argonDF['<I>Oxygenreal/mA']/m
argonDF

In [ ]:
#Spannungskorrektur mit Referenzelektrode
argonDF['Ewe_SHE/V']=argonDF['Ewe/V']+E_ref
argonDF

In [ ]:
#Spannungskorrektur ohmscher Widerstand
argonDF['Ewe_SHE,corr/V']=argonDF['Ewe_SHE/V']-argonDF['<I>Oxygenreal/mA']*E_ohm/1000
argonDF

In [ ]:
argonDF.to_csv('../data/KatalysatorAktivität.csv', index=False)